In [4]:
import sys
sys.path.append("/home/chaoqiy2/github/PyHealth-OMOP")

# 1. load dataset

In [5]:
from pyhealth.datasets import MIMIC3BaseDataset
base_ds = MIMIC3BaseDataset(root="/srv/local/data/physionet.org/files/mimiciii/1.4", flag="prod")

# 2. drug recommendation task

In [6]:
from pyhealth.tasks import DrugRecDataset
drugrec_ds = DrugRecDataset(base_ds)

----- preparing code mappings -----
source loaded from ./data cache
source loaded from ./data cache
source loaded from ./data cache
source loaded from ./data cache
mapping finished: RxNorm -> ATC4
mapping finished: ATC4 -> RxNorm
load time: 2.9381930828094482s
-----------------------------------------


100%|█████████████████████████████████████████████████████████████████████████| 46518/46518 [00:09<00:00, 5029.12it/s]


1. finish cleaning the dataset for drug recommendation task
2. tokenized the medical codes


In [4]:
from pyhealth.data.split import split_by_pat
train_loader, val_loader, test_loader = split_by_pat(drugrec_ds, [2/3, 1/6, 1/6], batch_size=64)

1. finish data splitting
2. generate train / val / test data loaders


# 3. use basic ML models

In [5]:
from pyhealth.models import MLModel
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
model = MLModel(
    task = "drug_recommendation",
    classifier = SVC(gamma='auto', verbose=1),
#     classifier = LogisticRegression(random_state=0, n_jobs=-1, max_iter=10, verbose=1),
    voc_size = drugrec_ds.voc_size,
    tokenizers = drugrec_ds.tokenizers
)

# 4. training

In [ ]:
from pyhealth.evaluator.evaluating_multilabel import evaluate_multilabel

model.fit(
    train_loader=train_loader,
    evaluate_fn=evaluate_multilabel,
    eval_loader=val_loader,
    monitor="jaccard",
)

[LibSVM]*..
*
optimization finished, #iter = 2077
obj = -1683.366462, rho = -0.912677
nSV = 2124, nBSV = 1389
Total nSV = 2124
[LibSVM]*
optimization finished, #iter = 24
obj = -7.992431, rho = -0.999244
nSV = 27, nBSV = 4
Total nSV = 27
[LibSVM]

# 5. evaluation

In [ ]:
evaluate_multilabel(model, test_loader, 'cpu')

In [ ]:
aa

# 3. initialize DL models

In [ ]:
from pyhealth.models import RETAIN
model = RETAIN(
    task = "drug_recommendation",
    voc_size = drugrec_ds.voc_size,
    tokenizers = drugrec_ds.tokenizers,
    emb_dim = 64,
)

# 4. traning

In [ ]:
from pyhealth.trainer import Trainer
from pyhealth.evaluator.evaluating_multilabel import evaluate_multilabel

trainer = Trainer(enable_logging=True, output_path="../output")
trainer.fit(model,
            train_loader=train_loader,
            epochs=50,
            evaluate_fn=evaluate_multilabel,
            eval_loader=val_loader,
            monitor="jaccard")

# load the best model
best_model = trainer.load_best(model)

# 5. evaluation

In [ ]:
evaluate_multilabel(model, test_loader, 'cpu')